In [11]:
import pandas as pd
import numpy as np 
import random
from sklearn.utils import shuffle 


mainData = pd.DataFrame(pd.read_csv("seattleWeather_1948-2017.csv"))
# mainData = mainData.sample(frac=1).reset_index(drop=True)
mainData = shuffle(mainData)
train,cv,test = np.split(mainData,[int(0.6*len(mainData)),int(0.8*len(mainData))])

#Features:
feature = train['TMIN']
# feature = feature/max(feature)
label = train['RAIN']
cvFeature = cv['TMIN']
# cvFeature = cvFeature/max(cvFeature)
cvLabel = cv['RAIN']
testFeature = test['TMIN']
# testFeature = testFeature/max(testFeature)
testLabel = test['RAIN']


label = label.replace([False,True],[0,1])
testLabel = testLabel.replace([False,True],[0,1])
cvLabel = cvLabel.replace([False,True],[0,1])

weights = 1

def sigmoid(weights,x):
    return (1/(1+np.exp(-(weights*x))))

def cost(weights,x,y):
    return np.mean((sigmoid(x,weights)-y)**2)

def derivative(weights,x,y):
    return np.mean((sigmoid(weights,x)-y)*x)

def gradientDescent(iterations,learningRate):
    global feature,label
    weightsOld = 1
    weightsNew = 0
    for i in range(0,iterations):
        weightsNew = weightsOld - learningRate*derivative(weightsOld,feature,label)
#         print("weightsOld:",weightsOld,"weightsNew:",weightsNew)
#         print("Subtract",(weightsOld-weightsNew))
        weightsOld = weightsNew
    return weightsOld

def test(weights,x,y):
    mse = cost(weights,x,y)
    accuracy = 100 - ((mse/np.mean(x))*100)
    print("Cost",mse)
    print("DataMean",np.mean(x))
    print("Accuracy",accuracy,"%")
    
    
def testDiscrete(weights,x,y):
    answers = []
    correct = 0
    for i in range(0,len(x)):
        pro = cost(weights,x.iloc[i],y.iloc[i])
        print(pro)
        if(pro>0.5 ):
            answers.append(1)
        else:
            answers.append(0)
    for i in range(0,len(answers)):
        if(answers[i]==testLabel.iloc[i]):
            correct = correct+1
            
    print("Answers",answers)
    print("Accuracy:",((correct/len(answers))*100))

#Learning Variables:
iterations = 500
lr = 0.001
learnedweightss = gradientDescent(iterations,lr)

#Testing:
print("Training Accuracy")
test(learnedweightss,feature,label)
print("----------------------")
print("CrossValidation Accuracy")
test(learnedweightss,cvFeature,cvLabel)
print("----------------------")
print("Testing Accuracy")
test(learnedweightss,testFeature,testLabel)
# print("Result Discrete Testing")
# testDiscrete(learnedweightss,testFeature,testLabel)

    
    


Training Accuracy
Cost 0.2423589520144161
DataMean 44.4795172863666
Accuracy 99.45512234214668 %
----------------------
CrossValidation Accuracy
Cost 0.24252423507449494
DataMean 44.608610567514674
Accuracy 99.45632865048007 %
----------------------
Testing Accuracy
Cost 0.2441815452645711
DataMean 44.523967912345924
Accuracy 99.45157281187227 %


In [10]:
import matplotlib.pyplot as plt
#Plotting Sigmoid:
sig = []

for i in range(0,len(testFeature)):
    sig.append(1/(1+np.exp((-1)*(testFeature.iloc[i]**1)*learnedweightss)))

plt.scatter(testFeature,sig)
plt.scatter(testFeature,testLabel,color='red')  
plt.show()